In [1]:
%load_ext autoreload
%autoreload 2

# Load EMA weights

In [2]:
from pathlib import Path
from omegaconf import OmegaConf

import torch
from plaid.diffusion import FunctionOrganismDiffusion
from plaid.denoisers import FunctionOrganismDiT, DenoiserKwargs
from plaid.constants import COMPRESSION_INPUT_DIMENSIONS, COMPRESSION_SHORTEN_FACTORS

device = torch.device("cuda")

In [3]:
model_id = "qfvl29in" 

ckpt_dir = Path("/data/lux70/plaid/checkpoints/plaid_compositional_conditioning") 
model_path = ckpt_dir / model_id / "last.ckpt"
config_path = ckpt_dir / model_id / "config.yaml"

cfg = OmegaConf.load(f"/data/lux70/plaid/checkpoints/plaid_compositional_conditioning/{model_id}/config.yaml")

In [4]:
compression_model_id = cfg['compression_model_id']
shorten_factor = COMPRESSION_SHORTEN_FACTORS[compression_model_id]
input_dim = COMPRESSION_INPUT_DIMENSIONS[compression_model_id]

In [5]:
denoiser_kwargs = cfg.denoiser
denoiser_kwargs.pop("_target_")

diffusion_kwargs = cfg.diffusion
diffusion_kwargs.pop("_target_")

'plaid.diffusion.FunctionOrganismDiffusion'

In [6]:
denoiser = FunctionOrganismDiT(**denoiser_kwargs, input_dim=input_dim)
diffusion = FunctionOrganismDiffusion(**diffusion_kwargs, model=denoiser, use_old_ema_module=True)

In [7]:
ckpt = torch.load(model_path)
# ckpt.keys()

In [8]:
ckpt.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters', 'ema_state_dict'])

In [9]:
diffusion.model.load_state_dict(ckpt['state_dict'])
diffusion.ema_model.load_state_dict(ckpt['ema_state_dict'])
_ = diffusion.eval().requires_grad_(False)
_ = diffusion.to(device)

In [10]:
# switch to EMA weights for inference
_ = diffusion.swap_to_ema()

In [11]:
from cheap.pretrained import load_model_from_id
cheap_model = load_model_from_id(compression_model_id)
_ = cheap_model.to(device)


from cheap.proteins import LatentToSequence,LatentToStructure
latent_to_sequence = LatentToSequence()
latent_to_sequence.to(device)

latent_to_structure = LatentToStructure()
latent_to_structure.to(device)

Using tanh layer at bottleneck...
Finished loading HPCT model with shorten factor 2 and 32 channel dimensions.


Creating ESMFold...
ESMFold model loaded in 37.91 seconds.


# Sample
Human, carbohydrate metabolic process:

In [27]:
# organism_idx = org_df[org_df.organism_id == "HUMAN"].organism_index.iloc[0]
# function_idx = go_df[go_df.GO_term == "carbohydrate metabolic process"].GO_idx.iloc[0]
# print(organism_idx, function_idx)

# organism_idx = 1326
# function_idx = 55


organism_idx, function_idx = None, None

In [13]:
device = torch.device("cuda")
diffusion = diffusion.to(device)

In [28]:
N, L = 32, 128
shape = (N, L, input_dim)
# organism_y_idxs = torch.full((N,), organism_idx)
# function_y_idxs = torch.full((N,), function_idx)

cond_scale = 10

diffusion.sampling_timesteps=1000
sampled_latent = diffusion.ddim_sample_loop(shape, organism_idx, function_idx, return_all_timesteps=True, cond_scale=cond_scale)

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: full() received an invalid combination of arguments - got (tuple, NoneType, dtype=torch.dtype, device=torch.device), but expected one of:
 * (tuple of ints size, Number fill_value, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, Number fill_value, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
print(sampled_latent.shape)
final_sample = sampled_latent[:, -1, :, :]
print(final_sample.shape)

print(final_sample.max(), final_sample.min())

In [ ]:
sampled_uncompressed = cheap_model.decode(final_sample, downsampled_mask=None)
print(sampled_uncompressed.min(), sampled_uncompressed.max())

In [29]:
# import pickle as pkl
# with open("test_sample.pkl", "wb") as f:
#     pkl.dump(sampled_uncompressed,f)

In [30]:
from cheap.utils import LatentScaler
latent_scaler = LatentScaler()
sampled_unscaled = latent_scaler.unscale(sampled_uncompressed) 

In [31]:
print(sampled_unscaled.shape)
print(sampled_unscaled.max(), sampled_unscaled.min())

torch.Size([32, 256, 1024])
tensor(2684.3967, device='cuda:0') tensor(-818.3135, device='cuda:0')


In [32]:
sequences = latent_to_sequence.to_sequence(sampled_unscaled)[-1]

In [33]:
sequences[:3]

['AIMNSVCRTEALPVMQPCPHGGCCKMVRLQAGAAPATLPRSLHEWDNTYQATTKAASTLRINVELSARCNFWPVNPFYKSCEMSMIPACGFPVPSGDCLRGQTFRQMDLCANYGEPVGTFSCTTDNNHWACVRPSAFTENVNGFRYTVRLRSGGEFLPAAEVPEDWTFLDIILSRDSGPWGFVIKMVDRANHSFISLAGIDQVRAYIRHGKDYGVGHGALCQVDSECNSSRFDLLVNRGAYPSFKHDYLGHLHEPI',
 'IEVHEAEQGDIIRKGQNESRCARLSKLIGLFYCNTKPPATLYLTNARALVVSQIRLALLHCSDCDCKTASDFEMYVTHGAPPDYLRKDFTFDSWEEWPTSDGGYNRLRFYVNRSNDTGVVRYALPGDMYPNTGYSFLVRWNDFPIPLLRYRLNISLTRDSLCVLKKEPTHMTDFCQDLELWSPPTSERRSFPSGVATVYEVTPYNAERSSLSVLGWRCEAIEFWVDAITNYTFDFTHHGEERAQVLGQESCRLYLN',
 'YRMALRSVAASVCNASTEDGCLQTTDSEDGRTTAEVYLGYLDTGLFDNGEANPQSAHLFVGCKSCSEFKCLNSGPYKSLPKLDESDGALDLTIPRAPDAEGDGCSVNNVTACIPAGSETWTESPMSDASFVQLFIPQVDSDLDGTLCKFDSMLDSVVVNLCMVGFAKTLPGKHCLFTVRRDLSRLSMPQPFHKARDCIVERSWGTGIHIYLNPSELGRVKVQITDEAHYKFEVVSANGTRFEVRVGEELEQPLYVD']

In [34]:
pdb_strs, raw_outputs = latent_to_structure.to_structure(sampled_unscaled, return_raw_outputs=True, sequences=sequences, batch_size=32)

(Generating structure): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:12<00:00, 132.01s/it]


In [35]:
raw_outputs['plddt'].mean(dim=-1).mean(dim=-1)

tensor([35.6189, 36.8377, 32.0829, 35.2335, 40.1130, 38.4485, 32.2962, 32.2980,
        33.0430, 39.5988, 30.5270, 47.9190, 39.8824, 37.1998, 40.4046, 34.3277,
        31.8439, 34.2220, 36.1206, 33.2253, 38.1927, 36.7022, 36.2579, 36.5120,
        31.6227, 34.5570, 36.5268, 33.7943, 31.4253, 37.9946, 34.8985, 46.9384])

In [36]:
import py3Dmol

# for i in range(len(pdb_strs)):
for i in range(0,10): 
    view = py3Dmol.view(width=600, height=600)
    view.addModelsAsFrames(pdb_strs[i])
    
    # Apply the plDDT color scheme
    view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}}})
    # view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    
    # # Add surface representation with plDDT-based color
    # view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}})
    view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}})

    view.zoomTo()
    view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [37]:
from plaid.evaluation import RITAPerplexity

perplexity_calc = RITAPerplexity(device=device)
perplexities = perplexity_calc.batch_eval(sequences)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
perplexities

19.32793426513672